<a href="https://colab.research.google.com/github/pavan581/earth-git/blob/master/beta_veg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
# Impotr and initialize Earth Engine
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-9hd2nrXAt0knr4lVncrdWvgOhihW4jnA3CdKJ-7g2I&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjZnkt6cTmoB2wYuxQ3GtEj7cw97CYCDiukiVNarqx2TysGP1oxjKE

Successfully saved authorization token.


In [15]:
!pip install geojson
!pip install geemap

     |████████████████████████████████| 1.9 MB 13.9 MB/s 
     |████████████████████████████████| 130 kB 48.1 MB/s 
     |████████████████████████████████| 1.3 MB 28.7 MB/s 
     |████████████████████████████████| 98 kB 6.4 MB/s 
     |████████████████████████████████| 8.7 MB 47.5 MB/s 
     |████████████████████████████████| 187 kB 39.7 MB/s 
     |████████████████████████████████| 99 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 42.6 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 
     |████████████████████████████████| 216 kB 55.9 MB/s 
     |████████████████████████████████| 1.7 MB 42.7 MB/s 
     |████████████████████████████████| 5.4 MB 26.3 MB/s 
     |████████████████████████████████| 95 kB 2.8 MB/s 
     |████████████████████████████████| 97 kB 7.5 MB/s 
     |████████████████████████████████| 340 kB 53.4 MB/s 
     |████████████████████████████████| 61 kB 6.9 MB/s 
     |██████████████████████

In [16]:
# GeoJSON to work with GeoJson Files
import geojson

# Libraries to plot maps
import matplotlib.pyplot as plt
import IPython.display as disp

import folium
from ipyleaflet import *
import geemap

%matplotlib inline

In [4]:
# Git Repository of the Project
!git clone https://github.com/pavan581/earth-git.git

Cloning into 'earth-git'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 22 (delta 0), reused 13 (delta 0), pack-reused 9
Unpacking objects: 100% (22/22), done.


# Data and Feature Import

In [22]:
with open('earth-git/2011_Dist.json') as f:
  data = geojson.load(f)

In [6]:
aoi = ee.FeatureCollection(data)
aoi

In [23]:


for dist in data['features']:
  if dist["properties"]["DISTRICT"] == 'East Godavari':
    aoi_data = dist
    break
  
aoi_coords = aoi_data["geometry"]["coordinates"]
aoi_alpha = ee.Geometry.Polygon(aoi_coords)

In [ ]:
sentinel2_data = ee.ImageCollection("COPERNICUS/S2_SR")
aoi = ee.FeatureCollection(data).filter("DISTRICT == 'EAST GODAVARI'")

In [ ]:
alpha_img = ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi_alpha)
    .filterDate(ee.Date('2020-01-01'), ee.Date('2020-05-30'))
    .first()
    .clip(aoi_alpha))

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Try 01',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#Define the visualization parameters.
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0.0,
  'max': 4000
}

folium.Map.add_ee_layer = add_ee_layer
a1_map = folium.Map()

a1_map.add_ee_layer(alpha_img, vizParams, 'AIO_ALPHA')

display(a1_map)

# Testing...

In [20]:
Map = geemap.Map()  # Add Google Satellite
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [41]:
%matplotlib inline
display(img.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5}))

In [38]:
%matplotlib inline
i = disp.Image(img.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5}), width=500)
i

/usr/local/lib/python3.7/dist-packages/IPython/core/formatters.py:364: FormatterWarning: image/png formatter returned invalid type <class 'tuple'> (expected (<class 'bytes'>, <class 'str'>)) for object: <IPython.core.display.Image object>
  FormatterWarning


<IPython.core.display.Image object>

In [24]:
img = ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi_alpha)
    .filterDate(ee.Date('2020-01-01'), ee.Date('2020-12-28'))
    .first()
    .clip(aoi_alpha))

vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0.0,
  'max': 4000
}

Map.addLayer(img, vizParams)

In [19]:
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Print the elevation of Mount Everest.
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = image.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Add Earth Engine layers to Map
Map.addLayer(image, vis_params, 'DEM')
Map.addLayer(xy, {'color': 'red'}, 'Mount Everest')

# Center the map based on an Earth Engine object or coordinates (longitude, latitude)
# Map.centerObject(xy, 4)
Map.setCenter(86.9250, 27.9881, 4)

Mount Everest elevation (m): 8729
